## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("weather_database/WeatherPy_Database.csv")
city_data_df.head()

,City ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Tocopilla,CL,2022-09-25 22:24:23,-22.0920,-70.1979,64.09,66,22,10.98,few clouds
1,1,Olafsvik,IS,2022-09-25 22:24:24,64.8945,-23.7142,45.07,62,1,10.96,clear sky
2,2,Arica,CL,2022-09-25 22:24:24,-18.4750,-70.3042,63.95,67,0,11.50,clear sky
3,3,Rikitea,PF,2022-09-25 22:24:24,-23.1203,-134.9692,69.04,59,0,15.50,clear sky
4,4,Barrow,US,2022-09-25 22:24:25,71.2906,-156.7887,32.02,80,100,14.97,light snow


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
6,6,Ghanzi,BW,2022-09-25 22:24:25,-21.5667,21.7833,75.20,20,0,11.56,clear sky
8,8,Avarua,CK,2022-09-25 22:24:26,-21.2078,-159.7750,78.85,73,75,13.80,broken clouds
9,9,Bartica,GY,2022-09-25 22:24:26,6.4000,-58.6167,82.72,93,68,1.72,broken clouds
16,16,Atuona,PF,2022-09-25 22:24:28,-9.8000,-139.0333,76.89,76,25,18.28,scattered clouds
17,17,Parrita,CR,2022-09-25 22:24:28,9.5201,-84.3225,81.54,77,100,4.18,overcast clouds
26,26,Alofi,NU,2022-09-25 22:24:31,-19.0595,-169.9187,75.09,94,100,3.44,overcast clouds
35,35,Bandarbeyla,SO,2022-09-25 22:24:33,9.4942,50.8122,77.22,83,87,25.17,overcast clouds
37,37,Medicine Hat,CA,2022-09-25 22:24:24,50.0501,-110.6683,78.85,28,40,9.22,scattered clouds
48,48,Mubi,NG,2022-09-25 22:31:48,10.2676,13.2644,75.52,74,83,3.15,broken clouds
49,49,Salalah,OM,2022-09-25 22:31:48,17.0151,54.0924,80.69,94,75,8.05,broken clouds


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City ID                203
City                   203
Country                203
Date                   203
Lat                    203
Lng                    203
Max Temp               203
Humidity               203
Cloudiness             203
Wind Speed             203
Current Description    203
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""

In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():

    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"    

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()   
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [9]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()

In [10]:
# 8a. Create the output File (CSV)
output_data_file = "vacation_search/WeatherPy_Vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [11]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [12]:
# 11a. Add a marker layer for each city to the map. 
max_temp = clean_hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))